In [1]:
import os
import json
import pandas as pd
import traceback

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(model="qwen2.5:3b")

In [23]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [24]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [25]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [26]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [27]:
quiz_generate_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

quiz_eval_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [28]:
quiz_chain = LLMChain(llm = llm, prompt=quiz_generate_prompt, output_key="quiz", verbose = True)

In [ ]:
review_chain = LLMChain(llm = llm, prompt = quiz_generate_prompt, output_key="review", verbose=True)

In [30]:
generate_eval_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [31]:
file_path = r"data.txt"
with open(file_path,'r') as file:
    TEXT = file.read()

In [32]:
NUMBER = 5
SUBJECT = "UML Designing"
TONE = "simple"

In [33]:
with get_openai_callback() as cb:
    response=generate_eval_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:What is UML? Everything You Need to Know About Unified Modeling Language
Diagrams for Software EngineeringSoftware Development
Back to top
What is UML?
UML, or Unified Modeling Language, is a visual modeling language that helps software developers visualize and construct new systems. 

It’s not a programming language — it’s a set of rules specifically for drawing diagrams. There are many types of software engineering diagrams, but think of this language like the software engineer’s version of a blueprint.

If you're brand new to UML, you might like our resource on UML Basics before digging in to UML diagrams. To learn more about UML diagramming, read on or jump ahead to one of the following sections:

Table of Contents
What is UML?
Video Intro to UML Diagramming
Common UML Notations
History of the Unified Modeling Language
What does a UML Diagram Do? Why Software Engineers Use the Unified Modeling Language
Start Diagramming With the Unified Modeling Langu

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:What is UML? Everything You Need to Know About Unified Modeling Language
Diagrams for Software EngineeringSoftware Development
Back to top
What is UML?
UML, or Unified Modeling Language, is a visual modeling language that helps software developers visualize and construct new systems. 

It’s not a programming language — it’s a set of rules specifically for drawing diagrams. There are many types of software engineering diagrams, but think of this language like the software engineer’s version of a blueprint.

If you're brand new to UML, you might like our resource on UML Basics before digging in to UML diagrams. To learn more about UML diagramming, read on or jump ahead to one of the following sections:

Table of Contents
What is UML?
Video Intro to UML Diagramming
Common UML Notations
History of the Unified Modeling Language
What does a UML Diagram Do? Why Software Engineers Use the Unified Modeling Language
Start Diagramming With the Uni

In [37]:
quiz=response.get("quiz")

In [46]:
# Get json block from the response which is in between ```json and ``` 
quiz = quiz.replace("### RESPONSE_JSON","")
json_block = json.loads(quiz)

In [48]:
for question in json_block:
	print(json_block[question]["mcq"])
	print(json_block[question]["options"])
	print(json_block[question]["correct"])

What is Unified Modeling Language (UML) primarily used for in software engineering?
{'a': 'To create actual executable code', 'b': 'As a programming language to write applications', 'c': 'For designing and visualizing system architectures', 'd': 'For managing project schedules and tasks'}
c
Which of the following is NOT a type of UML diagram mentioned in the text?
{'a': 'Activity Diagrams', 'b': 'Flowcharts', 'c': 'Class Diagrams', 'd': 'Component Diagrams'}
b
What does UML help software engineers visualize and construct new systems by?
{'a': 'Creating blueprints for buildings', 'b': 'Designing and structuring software components', 'c': 'Managing project schedules and tasks', 'd': 'Writing executable code'}
b
According to the text, who created UML in 1995 while working at Rational Software?
{'a': 'Grady Booch', 'b': 'Ivar Jacobson and James Rumbaugh', 'c': 'All of the above', 'd': 'None of the above'}
c
Which of the following is NOT a type of relationship shown in UML diagrams?
{'a': '

In [50]:
quiz_table_data = []
for key, value in json_block.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [51]:
quiz_table_data

[{'MCQ': 'What is Unified Modeling Language (UML) primarily used for in software engineering?',
  'Choices': 'a: To create actual executable code | b: As a programming language to write applications | c: For designing and visualizing system architectures | d: For managing project schedules and tasks',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is NOT a type of UML diagram mentioned in the text?',
  'Choices': 'a: Activity Diagrams | b: Flowcharts | c: Class Diagrams | d: Component Diagrams',
  'Correct': 'b'},
 {'MCQ': 'What does UML help software engineers visualize and construct new systems by?',
  'Choices': 'a: Creating blueprints for buildings | b: Designing and structuring software components | c: Managing project schedules and tasks | d: Writing executable code',
  'Correct': 'b'},
 {'MCQ': 'According to the text, who created UML in 1995 while working at Rational Software?',
  'Choices': 'a: Grady Booch | b: Ivar Jacobson and James Rumbaugh | c: All of the above | d: No

In [52]:
quiz=pd.DataFrame(quiz_table_data)

In [53]:
quiz.to_csv("gen_ai.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')